In [1]:
import numpy as np
import random
import decimal

# Toy Problem
t = np.array([[1,1,0,0,1,0], [1,0,0,1,1,1], [0,0,1,1,1,1], [1,1,1,1,1,1], [0,0,0,1,1,1], [1,1,1,1,0,0]])
sens = [3,2,2,1]
n = len(sens)

#sol = [(0,0), (0,4), (3,4), (4,2)]

# Predefined variables
gen_n = 100 # Number of generations
pop_n = 20  # Population size
mut_prob = 0.02 # Probability of mutation

In [2]:
#Checks if its valid the position
def check_position(sensor,pos,size):
    x = pos[0]+sensor-1
    y = pos[1]+sensor-1
    cont = True
    if(x > (size - 1) or y > (size - 1)):
        cont = False
    return cont

# gen_population(sensors, number of the population)
def gen_population(sens,n,size):
    population = []
    for i in range(n):
        localSol = []
        for se in sens:
            cont = True
            pos = (0,0)
            while cont:
                x = random.randint(0,size - 1)
                y = random.randint(0,size - 1)
                pos = (x,y)
                cont = not check_position(se,pos,size)
            localSol.append(pos)
        population.append(localSol)
    return population

gen_population(sens,10,5)

[[(2, 2), (0, 2), (1, 1), (0, 2)],
 [(0, 0), (3, 2), (2, 1), (2, 3)],
 [(1, 2), (3, 2), (3, 2), (4, 1)],
 [(2, 0), (0, 3), (2, 2), (3, 0)],
 [(1, 2), (0, 2), (0, 0), (4, 0)],
 [(0, 0), (0, 2), (0, 3), (3, 2)],
 [(1, 2), (2, 3), (0, 1), (0, 3)],
 [(0, 1), (3, 3), (3, 1), (1, 1)],
 [(0, 2), (1, 2), (0, 2), (4, 4)],
 [(1, 2), (1, 1), (2, 0), (2, 1)]]

In [3]:
# Generates the solution terrain gen_sol_t(solution, sensors)
def gen_sol_t(sol,sens,t,numbers=False):
    gen_sol = np.zeros_like(t)
    for i, sen in enumerate(sens):
        pos = sol[i]
        for x in range(pos[0],pos[0] + sen):
            for y in range(pos[1],pos[1] + sen):
                if numbers:
                    gen_sol[x][y] = i + 1
                else:
                    gen_sol[x][y] = 1
    return np.array(gen_sol)

# fev(terrain, sensors, solution)
def fev(t,sens,sol):
    return np.sum(np.multiply(t,gen_sol_t(sol,sens,t)))

sol = [(0,0), (0,4), (3,4), (4,2)]
fev(t,sens,sol)

11

In [4]:
class Individual(object):
    def __init__(self, sol, fitness):
        self.sol = sol
        self.fitness = fitness
    def __repr__(self):
        return '{}: {} {}'.format(self.__class__.__name__,
                                  self.sol,
                                  self.fitness)

def ranking(population):
    individuals = []
    for individual in population:
        fit = fev(t,sens,individual)
        individuals.append(Individual(individual,fit))
    sortedIndividuals = sorted(individuals, key=getKey, reverse=True)
    ranking_population = []
    for individual in sortedIndividuals:
        ranking_population.append(individual.sol)
    return ranking_population

def getKey(custom):
    return custom.fitness

def roulette(population):
    S = 0
    individuals = []
    for individual in population:
        fit = fev(t,sens,individual)
        individuals.append(Individual(individual,fit))
        S = S + fit
    sortedIndividuals = sorted(individuals, key=getKey, reverse=True)
    P = 0
    # Prevent S = 0 ALl parents are bad
    if S == 0:
        rndNum = random.randint(0,len(sortedIndividuals)-1)
        individual = sortedIndividuals[rndNum]
        return {"ind":individual,"pop":sortedIndividuals}
        R = fit = float(decimal.Decimal(random.randrange(0, 1*100))/100)
    else:
        R = fit = float(decimal.Decimal(random.randrange(0, S*100))/100)
    for individual in sortedIndividuals:
        P = P + individual.fitness
        if(P >= R):
            return {"ind":individual,"pop":sortedIndividuals}
    return sortedIndividuals[0]

def get_parents(population):
    cycle1 = roulette(population)
    parent1 = cycle1["ind"]
    population.remove(parent1.sol)
    cycle2 = roulette(population)
    #print(cycle2)
    parent2 = cycle2["ind"]
    return [parent1.sol,parent2.sol]

#pop = gen_population(sens,10,9)
#pop
#get_parents(pop)


In [5]:
# crossover(solution 1, solution 2, cross-over point) ** Crossover point starts at 1
def crossover(sol1,sol2,cop):
    new_sol1 = []
    new_sol2 = []

    for i in range(len(sol1)):
        if i < cop:
            new_sol1.append(sol1[i])
            new_sol2.append(sol2[i])
        else:
            new_sol1.append(sol2[i])
            new_sol2.append(sol1[i])
    return new_sol1, new_sol2
crossover([(1,1),(2,2)],[(3,4),(1,2)],1)

([(1, 1), (1, 2)], [(3, 4), (2, 2)])

In [6]:
def mutate(sol,prob,n):
    mutate_sol = np.array(sol)
    new_list = []
    for i in range(len(sol)):
        for j in range(2):
            if random.random() < prob:
                mutate_sol[i][j] = random.randint(0,n)
            else:
                pass
        cond = check_position(sens[i],tuple(mutate_sol[i]),n)
        if(cond):
            new_list.append(tuple(mutate_sol[i]))
        else:
            new_list.append(sol[i])
        #print(new_list)
    #tupla = tuple(mutate_sol)
    return new_list
sol = [(0,0), (0,4), (3,4), (4,2)]
mutate(sol, 0.3, 5)

[(0, 0), (0, 4), (3, 2), (2, 2)]

In [7]:
def disp_symbol(t,sol=False):
    if sol:
        if(t == 1 and sol != 0):
            return sol
        elif(t == 0 and sol != 0):
            return "#"
        else:
            return "-"
    else:
        if(t == 0):
            return "X"
        else:
            return "-"

def display_solution(t,sol,sens):
    n = len(t)
    print("  ",end='')
    for i in range(1,n +1):
        print(i,end=" ")
    print("               ",end='')
    for i in range(1,n +1):
        print(i,end=" ")
    print()
    for y in range(0,n):
        print(y+1,end=' ')
        for x in range(0,n):
            print(disp_symbol(t[y][x]),end=' ')
        if((y+1) == int(n/2)):
            print("     ==>     ",end='')
        else:
            print("             ",end='')
        print(y+1,end=' ')
        for x in range(0,n):
            print(disp_symbol(t[y][x],gen_sol_t(sol,sens,t,True)[y][x]),end=' ')
        print()
sol = [(0,0), (0,4), (3,4), (4,3)]
display_solution(t,sol,sens)

  1 2 3 4 5 6                1 2 3 4 5 6 
1 - - X X - X              1 1 1 # X 2 # 
2 - X X - - -              2 1 # # - 2 2 
3 X X - - - -      ==>     3 # # 1 - - - 
4 - - - - - -              4 - - - - 3 3 
5 X X X - - -              5 X X X 4 3 3 
6 - - - - X X              6 - - - - X X 


In [8]:
def run(t,sens,gen_n,pop_n,mut_prob,n,print_current=False):
    pop = gen_population(sens,pop_n,len(t))
    new_pop = []
    for i in range(gen_n):
        for j in range(int(pop_n/2)):
            parents = get_parents(pop)
            inds = crossover(parents[0],parents[1],random.randint(0,n))
            new_pop.append(mutate(inds[0],mut_prob,n))
            new_pop.append(mutate(inds[1],mut_prob,n))
        if print_current:
            print("Current Best: ",ranking(new_pop)[0],", FEV: ",fev(t,sens,ranking(new_pop)[0]))
            print()
            display_solution(t,ranking(new_pop)[0],sens)
        pop = new_pop
    print("Best: ",ranking(new_pop)[0],", FEV: ",fev(t,sens,ranking(new_pop)[0]))
    print()
    display_solution(t,ranking(new_pop)[0],sens)

#run(t,sens,gen_n,pop_n,mut_prob)

# Problems

Pop = 20| Mut 2% | Gen = 50

In [12]:
# Toy Problem
t = np.array([[1,1,0,0,1,0],
              [1,0,0,1,1,1],
              [0,0,1,1,1,1],
              [1,1,1,1,1,1],
              [0,0,0,1,1,1],
              [1,1,1,1,0,0]])
sens = [3,2,2,1]
n = len(sens)

# Predefined variables
gen_n = 50
pop_n = 20
mut_prob = 0.02

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(2, 3), (0, 0), (4, 2), (3, 2)] , FEV:  15

  1 2 3 4 5 6                1 2 3 4 5 6 
1 - - X X - X              1 2 2 X X - X 
2 - X X - - -              2 2 # X - - - 
3 X X - - - -      ==>     3 X X - 1 1 1 
4 - - - - - -              4 - - 4 1 1 1 
5 X X X - - -              5 X X # 3 1 1 
6 - - - - X X              6 - - 3 3 X X 


In [13]:
# Random Problem
t = np.array([[1,1,0,0,1,0,1,1,1],
              [1,0,0,1,1,1,1,1,1],
              [0,0,1,1,1,1,0,0,1],
              [1,1,1,1,1,1,1,1,1],
              [0,0,0,1,1,1,1,1,1],
              [1,1,1,1,0,0,1,1,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,1,1,1,0,0,0,1]])
sens = [3,2,2,3,1]
n = len(sens)

# Predefined variables
gen_n = 50
pop_n = 20
mut_prob = 0.02

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(3, 6), (5, 0), (0, 7), (2, 3), (3, 2)] , FEV:  27

  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 - - X X - X - - -              1 - - X X - X - 3 3 
2 - X X - - - - - -              2 - X X - - - - 3 3 
3 X X - - - - X X -              3 X X - 4 4 4 X X - 
4 - - - - - - - - -      ==>     4 - - 5 4 4 4 1 1 1 
5 X X X - - - - - -              5 X X X 4 4 4 1 1 1 
6 - - - - X X - - -              6 2 2 - - X X 1 1 1 
7 - - X - - X X X -              7 2 2 X - - X X X - 
8 - - X - - X X X -              8 - - X - - X X X - 
9 - - - - - X X X -              9 - - - - - X X X - 


In [23]:
# Solitaire Problem
t = np.array([[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0]])
sens = [2]
n = len(sens)

# Predefined variables
gen_n = 50
pop_n = 20
mut_prob = 0.02

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(4, 4)] , FEV:  1

  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 X X X X X X X X X              1 X X X X X X X X X 
2 X X X X X X X X X              2 X X X X X X X X X 
3 X X X X X X X X X              3 X X X X X X X X X 
4 X X X X X X X X X      ==>     4 X X X X X X X X X 
5 X X X X X X X X X              5 X X X X # # X X X 
6 X X X X X - - X X              6 X X X X # 1 - X X 
7 X X X X X - - X X              7 X X X X X - - X X 
8 X X X X X X X X X              8 X X X X X X X X X 
9 X X X X X X X X X              9 X X X X X X X X X 


In [26]:
# PERFECT MATCH
t = np.array([[1,1,1,0,0,0,0,0],
              [1,1,1,0,1,1,1,0],
              [1,1,1,0,1,1,1,0],
              [0,0,0,0,1,1,1,0],
              [0,0,0,1,0,0,0,0],
              [1,0,0,0,0,1,1,0],
              [0,0,0,0,0,1,1,0],
              [1,0,0,0,0,0,0,0]])
sens = [1,1,1,2,3,3]
n = len(sens)

# Predefined variables
gen_n = 50
pop_n = 20
mut_prob = 0.02
run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(3, 1), (0, 0), (1, 0), (1, 4), (1, 5), (0, 1)] , FEV:  16

  1 2 3 4 5 6 7 8                1 2 3 4 5 6 7 8 
1 - - - X X X X X              1 2 6 6 # X X X X 
2 - - - X - - - X              2 3 6 6 # 4 5 5 # 
3 - - - X - - - X              3 - 6 6 # 4 5 5 # 
4 X X X X - - - X      ==>     4 X # X X - 5 5 # 
5 X X X - X X X X              5 X X X - X X X X 
6 - X X X X - - X              6 - X X X X - - X 
7 X X X X X - - X              7 X X X X X - - X 
8 - X X X X X X X              8 - X X X X X X X 


In [18]:
# Big MATCH
t = np.array([[1,0,1,0,1,0,1,0,1,0],
              [0,1,1,0,0,1,1,0,1,1],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,1,1,1,0,0,0,1,1],
              [1,1,1,1,1,1,1,1,1,1],
              [0,0,1,1,1,0,1,0,0,1],
              [0,0,1,1,1,0,1,0,1,1],
              [0,1,0,1,0,1,1,1,0,1],
              [1,0,0,1,0,0,1,0,0,1],
              [1,0,0,0,0,0,0,0,0,0]])
sens = [5,4,3,2,1,1]
n = len(sens)

# Predefined variables
gen_n = 50
pop_n = 20
mut_prob = 0.02
run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(3, 1), (0, 6), (6, 6), (0, 1), (0, 2), (3, 4)] , FEV:  32

  1 2 3 4 5 6 7 8 9 10                1 2 3 4 5 6 7 8 9 10 
1 - X - X - X - X - X              1 - # 5 X - X 2 # 2 # 
2 X - - X X - - X - -              2 X 4 4 X X - 2 # 2 2 
3 X X X X X X X X X X              3 X X X X X X # # # # 
4 X X - - - X X X - -              4 X # 1 1 6 # # # 2 2 
5 - - - - - - - - - -      ==>     5 - 1 1 1 1 1 - - - - 
6 X X - - - X - X X -              6 X # 1 1 1 # - X X - 
7 X X - - - X - X - -              7 X # 1 1 1 # 3 # 3 - 
8 X - X - X - - - X -              8 X 1 # 1 # 1 3 3 # - 
9 - X X - X X - X X -              9 - X X - X X 3 # # - 
10 - X X X X X X X X X              10 - X X X X X X X X X 


Pop = 50| Mut 5% | Gen = 20

In [19]:
# Toy Problem
t = np.array([[1,1,0,0,1,0],
              [1,0,0,1,1,1],
              [0,0,1,1,1,1],
              [1,1,1,1,1,1],
              [0,0,0,1,1,1],
              [1,1,1,1,0,0]])
sens = [3,2,2,1]
n = len(sens)

# Predefined variables
gen_n = 20
pop_n = 50
mut_prob = 0.05

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(2, 3), (2, 1), (0, 4), (1, 0)] , FEV:  16

  1 2 3 4 5 6                1 2 3 4 5 6 
1 - - X X - X              1 - - X X 3 # 
2 - X X - - -              2 4 X X - 3 3 
3 X X - - - -      ==>     3 X # 2 1 1 1 
4 - - - - - -              4 - 2 2 1 1 1 
5 X X X - - -              5 X X X 1 1 1 
6 - - - - X X              6 - - - - X X 


In [20]:
# Random Problem
t = np.array([[1,1,0,0,1,0,1,1,1],
              [1,0,0,1,1,1,1,1,1],
              [0,0,1,1,1,1,0,0,1],
              [1,1,1,1,1,1,1,1,1],
              [0,0,0,1,1,1,1,1,1],
              [1,1,1,1,0,0,1,1,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,1,1,1,0,0,0,1]])
sens = [3,2,2,3,1]
n = len(sens)

# Predefined variables
gen_n = 20
pop_n = 50
mut_prob = 0.05

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(0, 6), (3, 7), (6, 0), (2, 3), (8, 6)] , FEV:  24

  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 - - X X - X - - -              1 - - X X - X 1 1 1 
2 - X X - - - - - -              2 - X X - - - 1 1 1 
3 X X - - - - X X -              3 X X - 4 4 4 # # 1 
4 - - - - - - - - -      ==>     4 - - - 4 4 4 - 2 2 
5 X X X - - - - - -              5 X X X 4 4 4 - 2 2 
6 - - - - X X - - -              6 - - - - X X - - - 
7 - - X - - X X X -              7 3 3 X - - X X X - 
8 - - X - - X X X -              8 3 3 X - - X X X - 
9 - - - - - X X X -              9 - - - - - X # X - 


In [22]:
# Solitaire Problem
t = np.array([[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0]])
sens = [2]
n = len(sens)

# Predefined variables
gen_n = 20
pop_n = 50
mut_prob = 0.05

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(6, 5)] , FEV:  2

  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 X X X X X X X X X              1 X X X X X X X X X 
2 X X X X X X X X X              2 X X X X X X X X X 
3 X X X X X X X X X              3 X X X X X X X X X 
4 X X X X X X X X X      ==>     4 X X X X X X X X X 
5 X X X X X X X X X              5 X X X X X X X X X 
6 X X X X X - - X X              6 X X X X X - - X X 
7 X X X X X - - X X              7 X X X X X 1 1 X X 
8 X X X X X X X X X              8 X X X X X # # X X 
9 X X X X X X X X X              9 X X X X X X X X X 


In [24]:
# PERFECT MATCH
t = np.array([[1,1,1,0,0,0,0,0],
              [1,1,1,0,1,1,1,0],
              [1,1,1,0,1,1,1,0],
              [0,0,0,0,1,1,1,0],
              [0,0,0,1,0,0,0,0],
              [1,0,0,0,0,1,1,0],
              [0,0,0,0,0,1,1,0],
              [1,0,0,0,0,0,0,0]])
sens = [1,1,1,2,3,3]
n = len(sens)

# Predefined variables
gen_n = 20
pop_n = 50
mut_prob = 0.05

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(1, 4), (4, 3), (3, 3), (4, 0), (2, 4), (0, 0)] , FEV:  18

  1 2 3 4 5 6 7 8                1 2 3 4 5 6 7 8 
1 - - - X X X X X              1 6 6 6 X X X X X 
2 - - - X - - - X              2 6 6 6 X 1 - - X 
3 - - - X - - - X              3 6 6 6 X 5 5 5 X 
4 X X X X - - - X      ==>     4 X X X # 5 5 5 X 
5 X X X - X X X X              5 # # X 2 # # # X 
6 - X X X X - - X              6 4 # X X X - - X 
7 X X X X X - - X              7 X X X X X - - X 
8 - X X X X X X X              8 - X X X X X X X 


In [25]:
# Big MATCH
t = np.array([[1,0,1,0,1,0,1,0,1,0],
              [0,1,1,0,0,1,1,0,1,1],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,1,1,1,0,0,0,1,1],
              [1,1,1,1,1,1,1,1,1,1],
              [0,0,1,1,1,0,1,0,0,1],
              [0,0,1,1,1,0,1,0,1,1],
              [0,1,0,1,0,1,1,1,0,1],
              [1,0,0,1,0,0,1,0,0,1],
              [1,0,0,0,0,0,0,0,0,0]])
sens = [5,4,3,2,1,1]
n = len(sens)

# Predefined variables
gen_n = 20
pop_n = 50
mut_prob = 0.05

run(t,sens,gen_n,pop_n,mut_prob,n)

Best:  [(2, 1), (4, 6), (7, 3), (3, 7), (1, 6), (0, 0)] , FEV:  32

  1 2 3 4 5 6 7 8 9 10                1 2 3 4 5 6 7 8 9 10 
1 - X - X - X - X - X              1 6 X - X - X - X - X 
2 X - - X X - - X - -              2 X - - X X - 5 X - - 
3 X X X X X X X X X X              3 X # # # # # X X X X 
4 X X - - - X X X - -              4 X # 1 1 1 # X # 4 - 
5 - - - - - - - - - -      ==>     5 - 1 1 1 1 1 2 4 4 2 
6 X X - - - X - X X -              6 X # 1 1 1 # 2 # # 2 
7 X X - - - X - X - -              7 X # 1 1 1 # 2 # 2 2 
8 X - X - X - - - X -              8 X - X 3 # 3 2 2 # 2 
9 - X X - X X - X X -              9 - X X 3 # # - X X - 
10 - X X X X X X X X X              10 - X X # # # X X X X 
